In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

# Compile bootstrap results to get point estimates and confidence intervals
For inverted analysis.

In [ ]:
bootstrap_probabilites = pd.read_csv('../results/sensitivity_analyses/bootstrap_subfertility.csv')

prob_summary = pd.DataFrame()

for dx in set(bootstrap_probabilites['diagnosis'].values):
    for col in bootstrap_probabilites.drop(columns=['iteration', 'diagnosis']):
        data = bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == dx, col].values

        mean = np.mean(data)
        se = stats.sem(data)

        ci = stats.norm.interval(alpha=0.95, loc=mean, scale=se)

        temp_prob = pd.DataFrame({'diagnosis': [dx],
                                  'metric': [col],
                                  'mean': [mean],
                                  'se': [se],
                                  'ci_low': [ci[0]],
                                  'ci_high': [ci[1]]})
        prob_summary = pd.concat([prob_summary, temp_prob])
prob_summary.to_csv('../results/sensitivity_analyses/probabilities_summary_subfertility.csv', index=False)

In [ ]:
# Forest plot of subfertility RR
sns.set_style("white")
sns.set_context("poster")

bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'normal', 'sort_key'] = 5
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'endo', 'sort_key'] = 1
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'fibroids', 'sort_key'] = 3
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'cysts', 'sort_key'] = 4
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'other', 'sort_key'] = 2

bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'normal', 'diagnosis'] = 'Normal Pelvis'
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'endo', 'diagnosis'] = 'Endometriosis'
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'fibroids', 'diagnosis'] = 'Uterine Fibroids'
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'cysts', 'diagnosis'] = 'Benign Ovarian Cysts'
bootstrap_probabilites.loc[bootstrap_probabilites['diagnosis'] == 'other', 'diagnosis'] = 'Other Gynecological Pathology'

bootstrap_probabilites = bootstrap_probabilites.sort_values(by='sort_key')

g = sns.catplot(x="rr", y="diagnosis", data=bootstrap_probabilites, kind="point",  
                join=False, ci=95, errwidth=1.5, capsize=0.2, color='black', height=7, aspect=2, scale=0.7)
plt.axvline(x=1, color="black", linestyle='dashed')
g.set_xlabels("Relative Risk of Subfertility", fontsize=20)
g.set_ylabels("")
plt.savefig('../results/sensitivity_analyses/subfertility_RR.png', bbox_inches='tight', dpi=300)
plt.show()